In [42]:
import polars as pl
import psycopg2
from dotenv import dotenv_values
from locus.utils.paths import PROJECT_ROOT, SQL_DIR
from tqdm import tqdm

In [43]:

config = dotenv_values(PROJECT_ROOT / ".env")

conn_local = psycopg2.connect(
    host=config["DB_HOST"],
    port=config["DB_PORT"],
    dbname=config["DB_NAME"],
    user=config["DB_USER"],
    password=config["DB_PASSWORD"],
)
cur_local = conn_local.cursor()

conn_hpc = psycopg2.connect(
    host=config["HPC_DB_HOST"],
    port=config["HPC_DB_PORT"],
    dbname=config["HPC_DB_NAME"],
    user=config["HPC_DB_USER"],
    password=config["HPC_DB_PASSWORD"],
)
cur_hpc = conn_hpc.cursor()

In [44]:
with open(SQL_DIR / "select_max_id.sql") as f:
    cur_local.execute(f.read())

# Retrieve query results
max_id = cur_local.fetchall()[0][0]
max_id

4233900

In [45]:
BATCH_SIZE = 1000

select_batch_string = "select * from dataset where id >= {} and id < {}"

df = pl.read_database(select_batch_string.format(0,  BATCH_SIZE), conn_local)
df

id,latitude,longitude,image
i64,f64,f64,object
1,41.906,12.455,<memory at 0x7f273e133640>
2,48.21107,16.36736,<memory at 0x7f273e01ab00>
3,43.942875,12.774091,<memory at 0x7f273e01ae00>
4,41.339054,14.507789,<memory at 0x7f273e01ad40>
5,-23.21027,-44.693222,<memory at 0x7f273e01aec0>
6,35.659386,139.70067,<memory at 0x7f273e01af80>
7,41.383595,2.167063,<memory at 0x7f273e01b040>
8,35.311726,46.96588,<memory at 0x7f273e01b100>
9,-23.385988,150.50623,<memory at 0x7f273e01b1c0>


In [48]:
for i in tqdm(range(0, max_id, BATCH_SIZE)):
    df = pl.read_database(select_batch_string.format(i, i + BATCH_SIZE), conn_local)
    df = df.drop("id")
    df.write_database(
        "dataset",
        f'postgresql://{config["HPC_DB_USER"]}:{config["HPC_DB_PASSWORD"]}@{config["HPC_DB_HOST"]}:{config["HPC_DB_PORT"]}/{config["HPC_DB_NAME"]}',
        if_table_exists="append",
    )

100%|██████████| 4234/4234 [8:33:16<00:00,  7.27s/it]  
